# Abstract

Many statistical populations contain distinct subpopulations, where each one may posses its own density. This can result in a multimodal distribution which can not be modelled by a single density. Modelling mixtures of distributions has key applications in document topic analysis and financial crisis modelling.

Gaussian Mixture Models, used to model subpopulations, are convex combinations of Gaussian densities. Parameters for this model can be estimated using the Expectation Maximization (EM) algorithm - a probablistic approach.

Recent discoveries in optimization research have yielded genetic algorithms inspired by processes found in nature. One particular example is Particle Swarm Optimization, which we implement to perform maximum likelihood estimation. We compare this algorithm to the classical EM by applying them to the Old Faithful dataset and to simulated mixture densities.

Particularly, we aim to compare this non-statistical approach (PSO) to the probablisitic approach (EM) to estimate densities for the univariate case where there are two distinct groups.


# Introduction

In practise, datasets are generated from process which can be represented as a combination of Gaussian distributions. The parameters of the distribution that generates these samples can offer insights into properties of the population, such as the subpopulation means and the weights of each mixture component. Concretely, this mixture distribution represents a decomposition of a single density into a convex combination of many. 

We formally define a Gaussian Mixture Model pdf $f$ with two components as

$$
f(x\space|\space\mu_{1},\sigma_{1}^{2},\mu_{2},\sigma_{2}^{2}) = \pi\phi(x\space|\space\mu_{1},\sigma_{1}^{2}) + (1-\pi)\phi(x\space|\space\mu_{2},\sigma_{2}^{2})
$$

where $\phi_{1},\phi_{2}$ are the Gaussian pdf and the weight $\pi \in [0,1]$.

This formulation describes two Gaussian components, each with a weight and their respective parameters $\mu$ and $\sigma_{2}$ being their mean and variance respectively.

We often do not know these parameters of the population given a sample dataset. At first glance, an approach would be to determine the log-likelihood function, yielding


$$
l(\mu_{1},\sigma_{1}^{2},\mu_{2},\sigma_{2}^{2}|x) = 
\sum_{i=1}^{n}[\ln(\pi\phi(x_{i}|\mu_{1},\sigma_{1}^{2}) + (1-\pi)\phi(x_{i}|\mu_{2},\sigma_{2}^{2}))]
$$

Maximum likelihood estimation by setting the score function to zero would require solving a multivariate optimization problem. This problem is challenging due to multiple variables, the complexity of the density $\phi$, and the summation of these. Thus numerical algorithms are required to find an approximate solution.


The most widely used approach is the Expectation Maximization (EM) algorithm [3]. It is a deterministic optimization technique derived from minorize-maximization (MM) methods. We say the log-likelihood function is maximized by by iteratively maximizing simpler functions. 

We propose to compare this to the Particle Swarm Optimization - a stochastic optimization algorithm first described by Kennedy [1] in 1995. It draws inspirations from behaviour seen in crowds of lifeforms such as school of fish or flocks of birds. The main idea is to begin with an initial 'swarm' of particles with values randomly chosen from the search space. Each particle represents a candidate solution to the optimization problem - each with its own 'inertia' which is a step size and direction. At each iteration the particles move in a direction determined by linear combination of its inertia and the global best solution found so far. After multiple iterations of this, the algorithm will terminate when the particles have converged to an optimum.

The comparison of the PSO and the EM will be the focus of the remainder of the paper. In the following section, we present the respective implementations of the methods. Then, we bring forth two ways of performing the compaarison, namely by applying our implementations to the well known Old Faithful dataset. To further quantify our comparison, we also perform Monte Carlo simulation. We end our report with the discussion of our results and potential for future improvements and research 


# Methodology

We begin by implementing our algorithms in Python. What follows is the mathematical description for each procedure. We make the important note that due to time constraints, we assume a priori knowledge of the true value for $\sigma^{2}$. For EM, we follow the formulas provided by Smyth [2] in his notes for finite mixture models:

We begin with an intial guess for the $\pi$, $\mu$, and $\sigma^{2}$ parameters.

The Expectation (E) step:

$$w_{i,1} = \frac{\pi\phi(x|\mu_{1},\sigma_{1}^{2})}{\pi\phi(x|\mu_{1},\sigma_{1}^{2}) +(1-\pi)\phi(x|\mu_{2},\sigma_{2}^{2})}$$

and the component membership weights for the second component are $w_{i,2} = 1 - w_{i,1}$.

Given the weights from the E step, we can update our estimates first by estimating the number of data points in the first component by $\hat{n_{1}} = \sum_{i=0}^{n}x_{i}w_{1,i}$, the weight of the first component by $\hat{\pi} = \frac{\hat{n_{1}}}{n}$, the mean of the first component $\hat{\mu_{1}} = \frac{1}{\hat{n}_{1}}\sum_{i=1}^{n}x_{i}w_{1,i}$, and the mean of the second component $\hat{\mu_{2}} = \frac{1}{n-\hat{n}_{1}}\sum_{i=1}^{n}x_{i}(1-w_{1,i})$. Note that the weight for the second component is $(1-\pi)$ and that we do not include the calculation for the variance as we are assumping this parameter is known due to time constraints.

The Particle Swarm Optimization algorithm is described as follows [1]:

1. Initialize $N$ particles, each with random velocity $\vec{v}_{i}$ and parameter values $\vec{p}_{i} =(\hat{\mu_{1}},\hat{\mu_{2}},\hat{\pi)}$.
2. Define each particle's fitness to be the log-likelihood function $l$ evaluated at those parameter values.
3. Define $\vec{g}_{best}$ to be the highest scoring position amongst all particles, and $\vec{p}_{best}$ to be each particle's personal best scoring position.
4. Update all particles using the following formula: $$ \vec{v}_{i+1} = \vec{v}_{i} + c_{1}r_{1}(\vec{p}_{best} - \vec{p}_{i}) + c_{2}r_{2}(\vec{p}_{best} - \vec{p}_{i})$$    
where $c_{1},c_{2}$ are constants and $r_{1},r_{2} \sim U(0,1)$
5. Repeat steps 2 to 4 until a termination condition is met.

With the implemented algorithms, we wish to evaluate them in two ways. First, we will run each algorithm on the dataset consisting of the eruption times of the Old Faithful geyser. Then we will use the estimated parameters to create a method to generate from this distribution and compare the plots of the dataset density to a plot of the density of a sample simulated from our method. To evaluate PSO we will compare the plot it produces with that of EM and the true dataset. The second method of evaluation is via Monte Carlo simulation. We will randomly generate multiple datasets from mixture distributions and run our algorithms on these simulated datasets. After running the simulation multiple times we will calculate the expected error between the estimate and the true parameter value, given by $E[|\hat{\theta} - \theta|]$. Please see the appendix for the R and Python code used for the experiments. In the next sections, we examine the results of our work and discuss the outcomes. 

To simplify our testing we imposed some restrictions to the parameters $(\mu_{1},\mu_{2},\pi)$ used to simulate the datasets. The ranges $\mu_{1} \in [0,10],\mu_{2} \in [11,20],\pi \in [0.61,0.89]$ were guaranteed to yield distributions with two distinct means and sufficient samples from each one.

# Results - Single Dataset

# Results - Multiple Simulated Datasets

# Discussion

The results of our analysis show that while PSO does sometimes make reasonable estimations for the parameters of the Gaussian mixture model, it currently does not beat EM. This is made clear by the higher mean absolute differences and by the graphical comparison. It is reasonable to conclude that the well known deterministic approach of EM is superior to the stochastic PSO. Of course, there is room for improvement in the implementation of the PSO algorithm. Future research could focus on spending more time tuning the conditions of the algorithm, such as the number of particles used, the number of iterations, as well as changing the weightings of the velocities. If exploring new possibilities for these values yields better results, then a next step would be to allow the algorithm to also estimate the variances of the components. Further enhancements would be to extend the algorithm to estimate a model with more than two components and to estimate multivariate rather than univarite models.

# References

[1] Kennedy, J. and Eberhart, R. (1995). Particle swarm optimization. Proceedings of IEEE International Conference on Neural Networks. IV. pp. 1942–1948.   
[2] Smyth, P. (2016). The EM algorithm for Gaussian Mixtures. Course notes - CS274A. University of California, Irvine.   
[3] Christian, R.P. and Casella, G. (2010). The EM Algorithm. In Introducing Monte Carlo Methods with R. New York, NY.   


# Appendix

## Appendix A - EM Implementation

```python
import math
import sys
import numpy as np
from matplotlib import pyplot as plt


def phi(x,mean,var):
	k = math.sqrt(2*math.pi*var)
	p = math.exp((x-mean)*(x-mean)/(-2*var))
	return p/k
def mix_dist(x,pi,mean1,mean2,var1,var2):
	comp1 = pi*phi(x,mean1,var1)
	comp2 = (1-pi)*phi(x,mean2,var2)
	return comp1+comp2
def gaussian_likelihood(x,pi,mean1,mean2,var1,var2):
	# x is a vector of data
	n = len(x)
	p = 0
	for i in range(0,n):
		p = p + math.log(mix_dist(x[i],pi,mean1,mean2,var1,var2)) 
	return p


def em(data,truSd,max_iter=20000):
	#mu1 = 0
	#mu2 = 0
	sd1 = 1#truSd#1
	sd2 = 1#truSd#1
	sprd = max(data) - min(data)
	mu1 = min(data)+0.25*sprd
	mu2 = min(data)+0.75*sprd
	pi1 = 0.5
	pi2 = 1 - pi1
	n = len(data)
	
	def prob(x,mu,sd):
		nator = np.vectorize(math.exp)(((x-mu)*(x-mu))/(-2*sd*sd))
		dator = math.sqrt(2*math.pi)*sd
		return (nator/dator)
	
	logL = gaussian_likelihood(data,pi1,mu1,mu2,sd1*sd1,sd2*sd2)
	
	for i in range(0,max_iter):
		p1 = prob(data,mu1,sd1)
		p2 = prob(data,mu2,sd2)
		w = (pi1*p1) / ((pi1*p1)+(pi2*p2))
		n1 = np.sum(w)
		n2 = n - n1
		pi1 = n1 / n
		pi2 = 1 - pi1

		mu1 = (1/n1)*np.sum(w*data)
		mu2 = (1/n2)*np.sum((1-w)*data)
		
		newLogL = gaussian_likelihood(data,pi1,mu1,mu2,sd1*sd1,sd2*sd2)
		
		if abs(newLogL-logL) < 0.0001:
			break
		
		logL = newLogL

		sd1 = math.sqrt(np.sum(w*(data-mu1)*(data-mu1)) / n1   )
		sd2 = math.sqrt((1/n2)*np.sum((1-w)*(data-mu2)*(data-mu2)))
		
	print(str(mu1))    
	print(str(pi1))
	print(str(sd1))
	print(str(mu2)) # 
	print(str(pi2)) # did not change
	print(str(sd2)) # correct



# Get parameters from call:
sed = int(sys.argv[1])
mu1 = float(sys.argv[2])
mu2 = float(sys.argv[3])
sd1 = float(sys.argv[4])
sd2 = float(sys.argv[5])
pi = float(sys.argv[6])
n1 = int(200*pi) 
n2 = 200-n1
	
np.random.seed(seed=sed)
x1 = np.random.normal(mu1,sd1,n1)
x2 = np.random.normal(mu2,sd2,n2)
x = np.append(x1,x2)

em(x,sd1)
            
```

## Appendix B - PSO Implementation

```python
import pandas as pd
import numpy as np
import sys
import math
from operator import attrgetter
from matplotlib import pyplot as plt

class Particle:
	def __init__(self,position,velocity):
		self.position = position
		self.velocity = velocity
		self.pbest = position
		self.current_fitness = 0
		self.best_fitness = 0
		self.num_parameters = len(self.position)
	def update_position(self,gbest,
						use_boundary=False,
						lower_bound=None,
						upper_bound=None):              
		c1 = 1
		c2 = 1
		r1 = np.random.rand(1)
		r2 = np.random.rand(1)
		delta = c1*r1*(self.pbest - self.position) + c2*r2*(gbest - self.position)
		
		upper_bound=np.zeros(self.num_parameters),
		lower_bound=np.zeros(self.num_parameters)
		
		cond1 = self.velocity + delta <= upper_bound
		cond2 = self.velocity + delta >= lower_bound
		
		if (use_boundary and cond1.all() and cond2.all()) or use_boundary==False:
			self.velocity = self.velocity + delta
			self.position = self.position + self.velocity
	def _phi(self,x,mean,var):
		k = math.sqrt(2*math.pi*var)
		p = math.exp((x-mean)*(x-mean)/(-2*var))
		return p/k
	def _mix_dist(self,x,pi,mean1,mean2,var1,var2):
		comp1 = pi*self._phi(x,mean1,var1)
		comp2 = (1-pi)*self._phi(x,mean2,var2)
		return comp1+comp2
	def _gaussian_likelihood(self,x,pi,mean1,mean2,var1,var2):
		# x is a vector of data
		n = len(x)
		p = 0
		for i in range(0,n):
			p = p + math.log(self._mix_dist(x[i],pi,mean1,mean2,var1,var2)) 
		return p
	def calculate_fitness(self,x,pi,mean1,mean2,var1,var2):
		self.current_fitness = self._gaussian_likelihood(x,pi,mean1,mean2,var1,var2)
		if self.current_fitness > self.best_fitness or self.best_fitness == 0:
			self.pbest = self.position
			self.best_fitness = self.current_fitness

			
def pso(x,truSig,truPi,num_particles=50,num_iter=100):
	N = num_particles
	iterations = num_iter

	data_min = min(x)
	data_max = max(x)
	lower_boundary = np.array([0,data_min,data_min,0,0])
	upper_boundary = np.array([1,data_max,data_max,10,10])    
	particles = [0] * N
	tolerance = 0.001

	# random intialization of particles

	np.random.seed(69)

	# estimating a weight parameter and mu parameter for each of the two gaussian components
	# posn[0] = weight
	# posn[1] = mu1
	# posn[2] = mu2

	for p in range(N):    
		rand_pi = np.random.uniform(0,1)
		rand_mean1 = np.random.uniform(data_min,data_max)
		rand_mean2 = np.random.uniform(data_min,data_max)
		sigma1 = truSig
		sigma2 = truSig
		rand_posn = np.array([rand_pi,rand_mean1,rand_mean2])
		rand_velocity = np.array([np.random.uniform(0,1),
								 np.random.uniform(0,1),
								 np.random.uniform(0,1)])
		
		particles[p] = Particle(rand_posn,rand_velocity)
		particles[p].calculate_fitness(x,rand_pi,rand_mean1,rand_mean2,sigma1,sigma2)

	gbest = max(particles,key=attrgetter('best_fitness'))    

	# repeat until convergence
	for i in range(iterations):
		for p in particles:
			p.update_position(gbest.position,True,lower_boundary,upper_boundary)

			params = p.position
			pi = params[0]
			mean1 = params[1]
			mean2 = params[2]
			sigma1 = truSig
			sigma2 = truSig
			p.calculate_fitness(x,pi,mean1,mean2,sigma1*sigma1,sigma2*sigma2)
			gbest = max(particles,key=attrgetter('best_fitness'))
	
	# Output results:
	if abs(truPi-gbest.pbest[0]) < abs(truPi-(1-gbest.pbest[0])):  
		print(gbest.pbest[1]) # Est mu1
		print(gbest.pbest[0]) # Est pi1
		print("1")	
		print(gbest.pbest[2]) # Est mu2
		print(1-gbest.pbest[0]) # Est pi2
		print("1")
	else:
		print(gbest.pbest[2]) # Est mu1
		print(1-gbest.pbest[0]) # Est pi1
		print("1")	
		print(gbest.pbest[1]) # Est mu2
		print(gbest.pbest[0]) # Est pi1
		print("1")
	
	#print(gbest.best_fitness)
	return 0

	
sed = int(sys.argv[1])
mu1 = float(sys.argv[2])
mu2 = float(sys.argv[3])
sd1 = float(sys.argv[4])
sd2 = float(sys.argv[5])
pi = float(sys.argv[6])
n1 = int(200*pi) 
n2 = 200-n1
	
np.random.seed(seed=sed)
x1 = np.random.normal(mu1,sd1,n1)
x2 = np.random.normal(mu2,sd2,n2)
x = np.append(x1,x2)

pso(x,sd1,pi)
```

## Appendix C - Old Faithful Analysis Code

```r
# Function to simulate dataset from a 
# two component gaussian mxiture model
rmixture <- function(n,mu1, mu2, sd1, sd2, pi) 
    u <- runif(n)
    r <- numeric(n)
    
    for(i in seq(1,n)) {
        if(u[i] <= pi) {
            r[i] = rnorm(1, mean = mu1, sd = sd1)
        } else {
            r[i] = rnorm(1, mean = mu2, sd = sd2)
        }
    }

    return(r)
}

# Initialize EM values
seed <- 23
mu1 <- 0
mu2 <- 0
sd1 <- 1
sd2 <- 1
pi <- 0.7
params <- paste(seed,mu1,mu2,sd1,sd2,pi)

x <- faithful[,2]
write.csv(x, "waittime.csv", row.names = FALSE)
d = density(x)
plot(d,lwd=4)

em_call <- paste("python faithfulEM.py",params)
emResults <- shell(em_call,intern=TRUE)
results <- emResults[1:(length(emResults))]
estMu1EM <- as.numeric(results[1])
estPi1EM <- as.numeric(results[2])
estSd1EM <- as.numeric(results[3])
estMu2EM <- as.numeric(results[4])
estPi2EM <- as.numeric(results[5])
estSd2EM <- as.numeric(results[6])

d = density(rmixture(272,estMu1EM,estMu2EM,estSd1EM,estSd2EM,estPi1EM )  )
lines(d,col=2,lwd=4,lty=3)

# Initialize PSO using sigmas from EM
seed <- 23
mu1 <- 0
mu2 <- 0
sd1 <- estSd1EM
sd2 <- estSd2EM
pi <- estPi1EM
params <- paste(seed,mu1,mu2,sd1,sd2,pi)

# True density of old faithful
x <- faithful[,2]
write.csv(x, "waittime.csv", row.names = FALSE)
d = density(x)
plot(d,lwd=4)

# Plot density generated from PSO
em_call <- paste("python faithfulPSO.py",params)
emResults <- shell(em_call,intern=TRUE)
results <- emResults[1:(length(emResults))]
estMu1PSO <- as.numeric(results[1])
estPi1PSO <- as.numeric(results[2])
estSd1PSO <- as.numeric(results[3])
estMu2PSO <- as.numeric(results[4])
estPi2PSO <- as.numeric(results[5])
estSd2PSO <- as.numeric(results[6])

d = density(rmixture(272,estMu1PSO,estMu2PSO,estSd1EM,estSd2EM,estPi1PSO )  )
lines(d,col=2,lty=3,lwd=4)   

```

## Appendix D - Monte Carlo Analysis Code

```r
# Monte Carlo simulation
M <- 1000

truMu1 <- numeric(M)
truPi1 <- numeric(M)
truSd1 <- numeric(M)
truMu2 <- numeric(M)
truPi2 <- numeric(M)
truSd2 <- numeric(M)

estMu1EM <- numeric(M)
estPi1EM <- numeric(M)
estSd1EM <- numeric(M)
estMu2EM <- numeric(M)
estPi2EM <- numeric(M)
estSd2EM <- numeric(M)

estMu1PSO <- numeric(M)
estPi1PSO <- numeric(M)
estSd1PSO <- numeric(M)
estMu2PSO <- numeric(M)
estPi2PSO <- numeric(M)
estSd2PSO <- numeric(M)

for(i in seq(1,M) ) {
    seed <- round(runif(n=1,min=1,max=100))
    mu1 <- round(runif(n=1,min=0,max=10),digits=2)
    mu2 <- round(runif(n=1,min=11,max=20),digits=2)
    sd1 <- round(runif(n=1,min=1,max=3),digits=2)
    sd2 <- sd1
    pi <- round(runif(n=1,min=0.61,max=0.89),digits=2)
    params <- paste(seed,mu1,mu2,sd1,sd2,pi)
   
    truMu1[i] <- mu1
    truMu2[i] <- mu2
    truPi1[i] <- pi 
    truPi2[i] <- 1 - pi
    truSd1[i] <- sd1
    truSd2[i] <- sd2
    
    # call python EM script
    em_call <- paste("python randomEM.py",params)
    emResults <- shell(em_call,intern=TRUE)
    results <- emResults[1:(length(emResults))]
    estMu1EM[i] <- as.numeric(results[1])
    estPi1EM[i] <- as.numeric(results[2])
    estSd1EM[i] <- as.numeric(results[3])
    estMu2EM[i] <- as.numeric(results[4])
    estPi2EM[i] <- as.numeric(results[5])
    estSd2EM[i] <- as.numeric(results[6])
    
    # call python PSO script
    pso_call <- paste("python randomPSO.py",params)
    psoResults <- shell(pso_call,intern=TRUE)
    results <- psoResults[1:(length(psoResults))]
    estMu1PSO[i] <- as.numeric(results[1])
    estPi1PSO[i] <- as.numeric(results[2])
    estSd1PSO[i] <- as.numeric(results[3])
    estMu2PSO[i] <- as.numeric(results[4])
    estPi2PSO[i] <- as.numeric(results[5])
    estSd2PSO[i] <- as.numeric(results[6])
}

print("EM")
mean(abs(truMu1 - estMu1EM))
mean(abs(truPi1 - estPi1EM))
mean(abs(truMu2 - estMu2EM))
mean(abs(truPi2 - estPi2EM))

print("PSO")
mean(abs(truMu1 - estMu1PSO ))
mean(abs(truPi1 - estPi1PSO ))
mean(abs(truMu2 - estMu2PSO ))
mean(abs(truPi2 - estPi2PSO ))

```